# Converts previous text format of data to json

In [8]:
import os
import json

In [3]:
def parse_metrics_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data = {"epochs": 0, "metrics": {}}
    metrics = data["metrics"]

    for line in lines:
        line = line.strip()

        # Parse epochs
        if line.startswith("EPOCHS:"):
            data["epochs"] = int(line.split(":")[1].strip())

        # Parse key-value metrics (e.g., macro_accuracy, micro_accuracy, etc.)
        elif ":" in line and not line.startswith("accuracy_per_class") and not line.startswith("f1_per_class") and not line.startswith("precision_per_class") and not line.startswith("recall_per_class") and not line.startswith("ROC-AUC_per_class_vs_rest"):
            key, value = line.split(":")
            metrics[key.strip()] = float(value.strip())

        # Parse metrics per class
        elif line.startswith("accuracy_per_class") or line.startswith("f1_per_class") or line.startswith("precision_per_class") or line.startswith("recall_per_class") or line.startswith("ROC-AUC_per_class_vs_rest"):
            key, values = line.split(":", 1)
            key = key.strip()
            class_metrics = {}
            
            class_metrics = values.split()
            results = dict()
            
            # Extract individual class metrics
            i = 0
            while i < len(class_metrics):
                class_name = ""
                
                while i < len(class_metrics) and not class_metrics[i].replace('.', '', 1).isdigit():
                    class_name = class_name + " " + class_metrics[i].removesuffix(":")
                    i += 1
                
                class_name = class_name.strip()
                if i < len(class_metrics):
                    class_value = class_metrics[i].strip()
                    results[class_name] = float(class_value)
                i += 1

            metrics[key] = results

    return data

In [6]:
def save_to_json(data, output_path):
    if not os.path.exists(os.path.dirname(output_path)):
        os.makedirs(os.path.dirname(output_path))
    with open(output_path, 'w') as json_file:
        json.dump(data, json_file, indent=2)

In [16]:
import glob

# find all files named "1.txt" in directory C:/VisualStudioRepositories/MUSIC_DATA/music_genre_classification/output/txt/ with recursion
file_paths = glob.glob("C:/VisualStudioRepositories/MUSIC_DATA/music_genre_classification/output/txt/**/1.txt", recursive=True)
file_paths[:5]

['C:/VisualStudioRepositories/MUSIC_DATA/music_genre_classification/output/txt\\fma_medium_binary\\model1_5_spacing\\0_epochs\\1.txt',
 'C:/VisualStudioRepositories/MUSIC_DATA/music_genre_classification/output/txt\\fma_medium_binary\\model1_5_spacing\\10_epochs\\1.txt',
 'C:/VisualStudioRepositories/MUSIC_DATA/music_genre_classification/output/txt\\fma_medium_binary\\model1_5_spacing\\15_epochs\\1.txt',
 'C:/VisualStudioRepositories/MUSIC_DATA/music_genre_classification/output/txt\\fma_medium_binary\\model1_5_spacing\\20_epochs\\1.txt',
 'C:/VisualStudioRepositories/MUSIC_DATA/music_genre_classification/output/txt\\fma_medium_binary\\model1_5_spacing\\25_epochs\\1.txt']

In [ ]:
# remove line "TEST results:" from all files
for file_path in file_paths:
    with open(file_path, 'r') as file:
        lines = file.readlines()
    with open(file_path, 'w') as file:
        for line in lines:
            if not line.startswith("TEST results:"):
                file.write(line)

In [17]:
# create .json for each file
for file_path in file_paths:
    data = parse_metrics_file(file_path)
    output_path = file_path.replace(".txt", ".json")
    save_to_json(data, output_path)

In [9]:
json_data = parse_metrics_file("C:/VisualStudioRepositories/MUSIC_DATA/music_genre_classification/output/txt/fma_medium_multi/model1_5_spacing/0_epochs/1.txt")
save_to_json(json_data, "C:/VisualStudioRepositories/MUSIC_DATA/music_genre_classification/output/txt/fma_medium_multi/model1_5_spacing/0_epochs/1.json")

In [18]:
# delete .txt files
for file_path in file_paths:
    os.remove(file_path)